<a href="https://colab.research.google.com/github/Sarinda251/CAP_5610/blob/master/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import svm
import statistics
import numpy as np
from numpy.random import seed
from numpy.random import randn
import matplotlib.pyplot as plt
seed(1)

## Preprocess Data

In [39]:
train_df = pd.read_csv('train.csv')
print(train_df.columns)


Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [40]:
sexToGender = {
  "male": 0,
  "female": 1
}

def embClean(x):
  if str(x) == 'nan':
    return 'S'
  return x

def fareClean(x, m):
  if str(x) == 'nan':
    return m
  return x

def fareBand(x):
  if x <= 7.91:
    return 0
  if x <= 14.454:
    return 1
  if x <= 31:
    return 2
  if x <= 512.329:
    return 3
  return 4

embBand = {
    "S": 0,
    "C": 1,
    "Q": 2
}

def ageClean(x, s, a):
  if str(x) == 'nan':
    return max(float(s*randn(1) + a),0)
  return float(x)

cleanedFare  = [x for x in train_df['Fare'].values if str(x) != 'nan']
mode = statistics.mode(cleanedFare)
cleanedAge = [x for x in train_df['Age'].values if str(x) != 'nan']
avg = statistics.mean(cleanedAge)
std = statistics.stdev(cleanedAge)

train_df['Age'] = train_df['Age'].transform(lambda x: ageClean(x, std, avg))
train_df['Fare'] = train_df['Fare'].transform(lambda x: fareClean(x, mode))
train_df['Fare'] = train_df['Fare'].transform(lambda x: fareBand(x))
train_df['Embarked'] = train_df['Embarked'].transform(lambda x: embClean(x))
train_df['Embarked'] = train_df['Embarked'].transform(lambda x: embBand[x])
train_df['Gender'] = train_df['Sex'].transform(lambda x: sexToGender[x])
train_df = train_df.drop(columns=["Ticket", "Cabin", "Name", "Sex"])

In [41]:
train_df._get_numeric_data()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,Gender
0,1,0,3,22.000000,1,0,0,0,0
1,2,1,1,38.000000,1,0,3,1,1
2,3,1,3,26.000000,0,0,1,0,1
3,4,1,1,35.000000,1,0,3,0,1
4,5,0,3,35.000000,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
886,887,0,2,27.000000,0,0,1,0,0
887,888,1,1,19.000000,0,0,2,0,1
888,889,0,3,49.040526,1,2,2,0,1
889,890,1,1,26.000000,0,0,2,1,0


In [42]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Age          891 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Fare         891 non-null    int64  
 7   Embarked     891 non-null    int64  
 8   Gender       891 non-null    int64  
dtypes: float64(1), int64(8)
memory usage: 62.8 KB


all null values have been filled, and columns have been turned into numerical datatypes.

In [43]:
train_x = train_df.drop(columns=["PassengerId", "Survived"])
train_y = train_df["Survived"]

In [44]:
train_x.describe()

,Pclass,Age,SibSp,Parch,Fare,Embarked,Gender
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,29.933108,0.523008,0.381594,1.508418,0.361392,0.352413
std,0.836071,14.286985,1.102743,0.806057,1.124141,0.635673,0.477990
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,20.869353,0.000000,0.000000,0.500000,0.000000,0.000000
50%,3.000000,29.000000,0.000000,0.000000,2.000000,0.000000,0.000000
75%,3.000000,38.841008,1.000000,0.000000,2.000000,1.000000,1.000000
max,3.000000,80.000000,8.000000,6.000000,4.000000,2.000000,1.000000


In [45]:
train_y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [46]:
train_x = train_x[["Fare", "Pclass", "Gender", "Age"]]
train_x

,Fare,Pclass,Gender,Age
0,0,3,0,22.000000
1,3,1,1,38.000000
2,1,3,1,26.000000
3,3,1,1,35.000000
4,1,3,0,35.000000
...,...,...,...,...
886,1,2,0,27.000000
887,2,1,1,19.000000
888,2,3,1,49.040526
889,2,1,0,26.000000


## Linear

In [47]:
clf = svm.SVC(kernel='linear')
scores = cross_val_score(clf, train_x, train_y, cv=5)
print(statistics.mean(scores))

0.7867365513778168


##Polynomial

In [48]:
clf2 = svm.SVC(kernel='poly', degree=2)
scores2 = cross_val_score(clf2, train_x, train_y, cv=5)
print(statistics.mean(scores2))

0.636375619860649


##RBF

In [49]:
clf3 = svm.SVC(kernel='rbf')
scores3 = cross_val_score(clf3, train_x, train_y, cv=5)
print(statistics.mean(scores3))

0.6363881740003766
